In [1]:
% load_ext sql

In [2]:
%%sql
sqlite:///CourseDataWarehouse.db

'Connected: @CourseDataWarehouse.db'

--Domain Integrity Queries
--(check that each column has sensible data types)

--Check to see that Max_Act (the maximum actual number of students in classes) is greater than or equal to Avg_Act (the average actual number of students in classes)

In [12]:
%%sql
SELECT Max_Act,Avg_Act,Program_Code
FROM CLASS_FACTS_W
    JOIN COURSE_W USING (CID)
    GROUP BY Program_Code
    LIMIT 10;

 * sqlite:///CourseDataWarehouse.db
Done.


MAX_ACT,AVG_ACT,PROGRAM_CODE
20,17.0,AC
25,22.9,AE
25,25.0,AH
0,1.0,AN
25,23.0,AR
20,23.0,AS
0,1.0,AY
0,6.0,BB
15,6.0,BEN
30,28.916666666666668,BI


--Check to see that see that faculty comes up as teaching in their departments classes

In [30]:
%%sql
SELECT LName,FName,Program_Name
FROM FACULTY_W
    JOIN CLASS_FACTS_W USING(FID)
    JOIN COURSE_W USING(CID)
GROUP BY Program_Name
LIMIT 30
;

 * sqlite:///CourseDataWarehouse.db
Done.


LNAME,FNAME,PROGRAM_NAME
Hartwell,Erica E.,Accounting
Govindaraja,Bama,American Studies
Alphonso,Gwendoline M.,Anthropology
Schwab,Katherine A.,Applied Ethics
Weinstein,Aaron Quinn,Arabic
Lawrence,Anna M.,Art History
Johnson,Jerelyn M.,Asian Studies
Campos,Javier F.,Bioengineering
Burke,Gregory Patrick,Biology
D'Agostin,Joseph R.,Black Studies


--Note that the above is INCORRECT - not a one of the faculty members listed appears to be associated with the right department...but we are out of time - so we cannot go back and debug this issue! The error gave us a much-needed chuckle though - e.g., Prof. Stott is listed as teaching "family therapy" - since he's a priest, there's some irony there!

--Entity Integrity Queries
--(check that the number of records in each table makes sense)

In [31]:
%%sql
SELECT COUNT(LName), COUNT(FName)
FROM FACULTY_W;

 * sqlite:///CourseDataWarehouse.db
Done.


COUNT(LName),COUNT(FName)
1096,1096


In [32]:
%%sql
SELECT DISTINCT COUNT(LName)
FROM FACULTY_W;

 * sqlite:///CourseDataWarehouse.db
Done.


COUNT(LName)
1096


In [33]:
%%sql
SELECT COUNT(crn)
FROM COURSE_W;

 * sqlite:///CourseDataWarehouse.db
Done.


COUNT(crn)
6069


In [35]:
%%sql
SELECT COUNT(Credits)
FROM COURSE_W;

 * sqlite:///CourseDataWarehouse.db
Done.


COUNT(Credits)
6069


In [ ]:
--Houston; we have a problem. There should be 4423 courses in the Course_W table
(i.e., the table from the Catalog). Again, there is no time to debug this error!

In [40]:
%%sql
SELECT COUNT(Year)
FROM TERM_W;

 * sqlite:///CourseDataWarehouse.db
Done.


COUNT(Year)
19


In [41]:
%%sql
SELECT COUNT(Term)
FROM TERM_W;

 * sqlite:///CourseDataWarehouse.db
Done.


COUNT(Term)
19


In [42]:
%%sql
SELECT COUNT(Building)
FROM LOCATION_W;

 * sqlite:///CourseDataWarehouse.db
Done.


COUNT(Building)
206


In [43]:
%%sql
SELECT COUNT(Room)
FROM LOCATION_W;

 * sqlite:///CourseDataWarehouse.db
Done.


COUNT(Room)
206


In [47]:
%%sql
SELECT COUNT(Cap)
FROM CLASS_FACTS_W;

 * sqlite:///CourseDataWarehouse.db
Done.


COUNT(Cap)
2040


In [45]:
%%sql
SELECT COUNT(Act)
FROM CLASS_FACTS_W;

 * sqlite:///CourseDataWarehouse.db
Done.


COUNT(Act)
2040


And here we have another issue - this table should have 15,937 records. Clearly there is another problem we need time to debug.

--Relational Integrity Queries
--(check that the FKs are join compatible w/PKs; make sure mandatory relationships fulfill NOT NULL constraint)

--Make sure Accounting courses are taught mainly in DSB

In [75]:
%%sql
SELECT Program_Name,Building
FROM COURSE_W
    JOIN CLASS_FACTS_W USING(CID)
    JOIN LOCATION_W USING(LID)
WHERE Program_Name='Accounting'
GROUP BY BUILDING
LIMIT 20;

 * sqlite:///CourseDataWarehouse.db
Done.


PROGRAM_NAME,BUILDING
Accounting,CNS
Accounting,DSB


--Make sure Count of Courses for faculty is about 3-4 per term, which is the typical number in faculty contracts.

In [74]:
%%sql
SELECT DISTINCT Term,LName,FName,Count_of_Courses
FROM TERM_W
    JOIN CLASS_FACTS_W USING(TermID)
    JOIN FACULTY_W USING(FID)
LIMIT 20;

 * sqlite:///CourseDataWarehouse.db
Done.


TERM,LNAME,FNAME,COUNT_OF_COURSES
Fall2014,Leclair,Mark S.,19
Fall2015,Leclair,Mark S.,19
Fall2016,Leclair,Mark S.,19
Fall2017,Leclair,Mark S.,19
Fall2018,Leclair,Mark S.,19
Fall2014,Aksan,Anna-Maria,19
Fall2015,Aksan,Anna-Maria,19
Fall2016,Aksan,Anna-Maria,19
Fall2017,Aksan,Anna-Maria,19
Fall2018,Aksan,Anna-Maria,19


--And we have another issue - the data clearly are not right! (Todd Martin is a Professor of the Practice and s/b at 4 whereas the others are Tenure-track faculty and s/b at 3...)

--Want to see that Dawn W Massey comes up as teaching AC classes

In [76]:
%%sql
SELECT LName,FName,Program_Code
FROM FACULTY_W
    JOIN CLASS_FACTS_W USING(FID)
    JOIN COURSE_W USING(CID)
WHERE LName='Massey' and FName='Dawn W'
;

 * sqlite:///CourseDataWarehouse.db
Done.


LNAME,FNAME,PROGRAM_CODE


--Note that the above is NOT correct...We are SURE Dawn W Massey has taught in two programs (i.e., AC and BU).